# Arquitectura de Streaming en tiempo real. 
#### Autores: Yahya El Baroudi, Samuel Corrionero, Ismael González y Jairo Farfán. 

In [30]:
import time

print("🏗️ Levantando infraestructura Docker...")
!docker compose -f docker/docker-compose.yml up -d

print("⏳ Esperando 30 segundos a que Kafka arranque...")
time.sleep(30) # Damos tiempo a que "caliente"
print("✅ Infraestructura lista.")

🏗️ Levantando infraestructura Docker...
[+] Running 0/1
 ⠋ Network docker_default  Creating                                        0.1s 
[+] Running 0/1
 ⠋ Network docker_default  Creating                                        0.1s 
[+] Running 2/3
 ✔ Network docker_default  Created                                         0.1s 
 ✔ Container zookeeper     Created                                         0.1s 
 ⠋ Container kafka         Creating                                        0.0s 
[+] Running 2/3
 ✔ Network docker_default  Created                                         0.1s 
 ✔ Container zookeeper     Created                                         0.1s 
 ⠋ Container kafka         Creating                                        0.0s 
[+] Running 2/3
 ✔ Network docker_default  Created                                         0.1s 
 ⠙ Container zookeeper     Starting                                        0.2s 
 ✔ Container kafka         Created                                    

In [31]:

import subprocess

print("twitter_simulator: 🚀 Iniciando simulador de tweets en segundo plano...")
# Esto lanza el script sin bloquear la celda
productor_process = subprocess.Popen(["python", "src/productor.py"])
print(f"✅ Productor corriendo (PID: {productor_process.pid})")

twitter_simulator: 🚀 Iniciando simulador de tweets en segundo plano...
✅ Productor corriendo (PID: 35624)


🔄 Iniciando simulador de Twitter hacia localhost:9092...
📩 Enviando: qué complicado es configurar #Streaming
📩 Enviando: qué complicado es configurar #Streaming
📩 Enviando: estoy aprendiendo mucho con #IA
📩 Enviando: estoy aprendiendo mucho con #IA
📩 Enviando: repasando conceptos de #Examen
📩 Enviando: repasando conceptos de #Examen


In [32]:
# Celda de código
import os
import sys
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, current_timestamp, window, count, max as max_
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import pyspark

# Configuración del entorno
spark_version = pyspark.__version__.split("+")[0]
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_2.12:{spark_version} pyspark-shell'

output_dir = Path("data")
output_dir.mkdir(exist_ok=True)

# 1. Iniciar Spark
spark = SparkSession.builder \
    .appName("JupyterStreaming") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# 2. Lectura
df_raw = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "tweets_topic") \
    .option("startingOffsets", "latest") \
    .load()

# 3. Esquema
schema = StructType([
    StructField("usuario", StringType(), True),
    StructField("texto", StringType(), True),
    StructField("hashtag_principal", StringType(), True),
    StructField("timestamp", DoubleType(), True),
])

# 4. Procesamiento
df_parsed = df_raw.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), schema).alias("data")) \
    .select("data.*") \
    .where(col("hashtag_principal").isNotNull()) \
    .withColumn("ts", current_timestamp())

# 5. Agregación (Ventana de 1 minuto)
conteo = df_parsed \
    .withWatermark("ts", "1 minutes") \
    .groupBy(window(col("ts"), "60 seconds"), col("hashtag_principal")) \
    .agg(count("*").alias("total")) \
    .orderBy(col("total").desc())

# 6. FUNCIÓN DE VISUALIZACIÓN PARA JUPYTER

def mostrar_en_jupyter(batch_df, batch_id):
    if batch_df.isEmpty():
        return
    pdf = batch_df.limit(10).toPandas()
    print(f"📊 Actualización del Stream - Batch ID: {batch_id}")
    csv_path = output_dir / f"batch_{batch_id}.csv"
    pdf.to_csv(csv_path, index=False)
    print(f"Guardado en {csv_path}")
    print(pdf.to_string(index=False))

# 7. Ejecutar Stream
print("📡 Escuchando... La tabla aparecerá abajo 👇")
query = conteo.writeStream \
    .outputMode("complete") \
    .foreachBatch(mostrar_en_jupyter) \
    .trigger(processingTime="5 seconds") \
    .start()

# NOTA: Esta celda se quedará ejecutando "para siempre" (verás un asterisco [*]).
# Para pararla, pulsa el botón de "Stop" (cuadrado) en el menú de Jupyter.
query.awaitTermination()

📩 Enviando: mañana tengo examen de #IA
📡 Escuchando... La tabla aparecerá abajo 👇
📩 Enviando: increíble la velocidad de #Examen
📩 Enviando: increíble la velocidad de #Examen
📩 Enviando: mañana tengo examen de #Spark
📩 Enviando: mañana tengo examen de #Spark
📩 Enviando: mañana tengo examen de #IA
📩 Enviando: mañana tengo examen de #IA
📩 Enviando: estoy aprendiendo mucho con #Kafka
📩 Enviando: estoy aprendiendo mucho con #Kafka
📩 Enviando: estoy aprendiendo mucho con #IA
📩 Enviando: estoy aprendiendo mucho con #IA
📩 Enviando: increíble la velocidad de #Python
📩 Enviando: increíble la velocidad de #Python
📩 Enviando: estoy aprendiendo mucho con #BigData
📩 Enviando: estoy aprendiendo mucho con #BigData
📩 Enviando: repasando conceptos de #Examen
📩 Enviando: repasando conceptos de #Examen
📩 Enviando: estoy aprendiendo mucho con #Streaming
📩 Enviando: estoy aprendiendo mucho con #Streaming
📩 Enviando: estoy aprendiendo mucho con #RealTime
📩 Enviando: estoy aprendiendo mucho con #RealTime
📩 En

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/cofer/anaconda3/envs/arqesp/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cofer/anaconda3/envs/arqesp/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cofer/anaconda3/envs/arqesp/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt



🛑 Simulador detenido por el usuario.


KeyboardInterrupt: 

In [33]:
# Celda de código
# Ejecuta esto SOLO cuando hayas parado la celda anterior
print("🛑 Apagando el productor...")
productor_process.kill()


🛑 Apagando el productor...


### 📂 Visualización de batches grabados
Los archivos CSV generados por cada batch se guardan en `data/` con el nombre `batch_<runId>.csv`. La siguiente celda lista esos archivos y los carga con pandas para que puedas inspeccionar los resultados después de parar el stream.

In [34]:
from pathlib import Path
import pandas as pd

print("📂 Batches guardados:")
for path in sorted(Path("data").glob("batch_*.csv")):
    print(f"- {path.name}")
    display(pd.read_csv(path))

📂 Batches guardados:
- batch_1.csv


,window,hashtag_principal,total
0,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#Spark,1


- batch_10.csv


,window,hashtag_principal,total
0,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Examen,11
1,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#IA,10
2,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Spark,10
3,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#BigData,9
4,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Streaming,8
5,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#RealTime,7
6,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Kafka,7
7,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Python,6
8,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#Kafka,3
9,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#IA,3


- batch_11.csv


...

,window,hashtag_principal,total
0,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Examen,10
1,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#BigData,7
2,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#IA,7
3,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Streaming,6
4,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Kafka,6
5,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#RealTime,5
6,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Spark,5
7,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Python,4
8,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#Kafka,3
9,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#IA,3


- batch_8.csv


,window,hashtag_principal,total
0,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Examen,11
1,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#BigData,8
2,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Streaming,8
3,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#IA,8
4,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Kafka,6
5,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Spark,6
6,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Python,6
7,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#RealTime,5
8,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#Kafka,3
9,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#IA,3


- batch_9.csv


,window,hashtag_principal,total
0,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Examen,11
1,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#IA,9
2,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#BigData,8
3,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Streaming,8
4,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#RealTime,7
5,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Kafka,7
6,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Spark,7
7,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Python,6
8,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#Kafka,3
9,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#IA,3


In [35]:

print("🛑 Apagando infraestructura...")
!docker compose -f docker/docker-compose.yml down

🛑 Apagando infraestructura...
[+] Running 0/1
 ⠋ Container kafka  Stopping                                               0.1s 
[+] Running 0/1
 ⠋ Container kafka  Stopping                                               0.1s 
[+] Running 0/1
 ⠙ Container kafka  Stopping                                               0.2s 
[+] Running 0/1
 ⠙ Container kafka  Stopping                                               0.2s 
[+] Running 0/1
 ⠹ Container kafka  Stopping                                               0.3s 
[+] Running 0/1
 ⠹ Container kafka  Stopping                                               0.3s 
[+] Running 0/1
 ⠸ Container kafka  Stopping                                               0.4s 
[+] Running 0/1
 ⠸ Container kafka  Stopping                                               0.4s 
[+] Running 0/1
 ⠼ Container kafka  Stopping                                               0.5s 
[+] Running 0/1
 ⠼ Container kafka  Stopping                                               0.5s 


25/12/15 18:44:03 ERROR MicroBatchExecution: Query [id = d0068d75-9763-4318-8c6a-f65a68286287, runId = 4f546360-88a1-4ca3-87d6-46eaefb32fd7] terminated with error
java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.TimeoutException: Timed out waiting for a node assignment. Call: describeTopics
	at java.base/java.util.concurrent.CompletableFuture.reportGet(CompletableFuture.java:396)
	at java.base/java.util.concurrent.CompletableFuture.get(CompletableFuture.java:2073)
	at org.apache.kafka.common.internals.KafkaFutureImpl.get(KafkaFutureImpl.java:165)
	at org.apache.spark.sql.kafka010.ConsumerStrategy.retrieveAllPartitions(ConsumerStrategy.scala:66)
	at org.apache.spark.sql.kafka010.ConsumerStrategy.retrieveAllPartitions$(ConsumerStrategy.scala:65)
	at org.apache.spark.sql.kafka010.SubscribeStrategy.retrieveAllPartitions(ConsumerStrategy.scala:102)
	at org.apache.spark.sql.kafka010.SubscribeStrategy.assignedTopicPartitions(ConsumerStrategy.scala:113)
	at org.apache.sp